# C1 W2 Group 8 - Task 1


In [ ]:
from PIL import Image
from tqdm import tqdm
import cv2
import numpy as np
from src.data import AUGMENTATIONS_QSD1_W4_LIST, FRAMES_QSD1_W4_LIST, GT_QSD1_W4_LIST
from src.paths import BBDD_PATH, QSD1_W4_PATH, QSD1_NON_AUGMENTED_W4_PATH, WEEK_4_RESULTS_PATH

In [ ]:
BBDD_PATH_LIST = sorted(BBDD_PATH.glob("*.jpg"))
QSD1_W4_PATH_LIST = sorted(QSD1_W4_PATH.glob("*.jpg"))
QSD1_NON_AUGMENTED_W4_PATH_LIST = sorted(QSD1_NON_AUGMENTED_W4_PATH.glob("*.jpg"))

In [ ]:
database_image_PIL_list = [Image.open(db_img_path) for db_img_path in BBDD_PATH_LIST]  # Load once
for idx, db_img in enumerate(database_image_PIL_list):
    assert db_img.filename.endswith(f"{idx}.jpg")

In [ ]:
query_d1_image_PIL_list = [Image.open(query_img_path) for query_img_path in QSD1_W4_PATH_LIST]  # Load once
for idx, query_img in enumerate(query_d1_image_PIL_list):
    assert query_img.filename.endswith(f"{idx}.jpg")

In [ ]:
non_augmented_d1_image_PIL_list = [Image.open(query_img_path) for query_img_path in QSD1_NON_AUGMENTED_W4_PATH_LIST]  # Load once
for idx, query_img in enumerate(non_augmented_d1_image_PIL_list):
    assert query_img.filename.endswith(f"{idx}.jpg")

In [ ]:
for file, augmentation, frames, gt in zip(QSD1_W4_PATH_LIST, AUGMENTATIONS_QSD1_W4_LIST, FRAMES_QSD1_W4_LIST, GT_QSD1_W4_LIST):
    print(f"File={file.stem}, Aug={augmentation}, Frames={frames}, GT={gt}")

## Denoising over the images

In [ ]:
from src.denoising import denoise_image
import matplotlib.pyplot as plt

In [ ]:
denoised_query_d1_PIL_list = [denoise_image(image) for image in query_d1_image_PIL_list]

In [ ]:
def plot_images_with_multiple_filters(original_images, noisy_images, denoised_images):
    num_images = len(original_images)
    num_cols = 3
    fig, axes = plt.subplots(num_images, num_cols, figsize=(25, 5 * num_images))  # +1 for original image

    for i in range(num_images):
        original_image = original_images[i]
        noisy_image = noisy_images[i]

        axes[i, 0].imshow(original_image)
        axes[i, 0].set_title(f'Original Image {i+1}')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(noisy_image)
        axes[i, 1].set_title(f'Noisy image Image {i+1}')
        axes[i, 1].axis('off')

        axes[i, 2].imshow(denoised_images[i])
        axes[i, 2].set_title(f'Denoised {i+1}')
        axes[i, 2].axis('off')

    plt.tight_layout()
    plt.savefig(WEEK_4_RESULTS_PATH / "Task_1" / 'denoised_images_plot.png')
    plt.close()

In [ ]:
#plot_images_with_multiple_filters(non_augmented_d1_image_PIL_list, query_d1_image_PIL_list, denoised_query_d1_PIL_list)

## Background removal

In [ ]:
from src.background import get_painting_masks, crop_image_by_mask

def crop_paintings_from_image(image, ground_truth):
    """
    Crop multiple paintings from a single image based on ground truth points.

    Args:
        image (PIL.Image): The input image to crop.
        ground_truth (list): List containing ground truth data where each entry is
                             a list with a score and a list of four corner points.

    Returns:
        list: A list of cropped images (as PIL.Images) corresponding to each painting.
    """
    # Convert PIL image to OpenCV format (numpy array)
    image_np = np.array(image)
    cropped_images = []

    for entry in ground_truth:
        # Extract the corner points for the painting
        points = entry[1]

        # Convert points to np.float32 for OpenCV compatibility
        src_points = np.array(points, dtype=np.float32)

        # Define the destination points for the transformed rectangle
        # Compute the width and height based on the distances between points
        width = int(np.linalg.norm(src_points[1] - src_points[0]))
        height = int(np.linalg.norm(src_points[2] - src_points[1]))
        dst_points = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)

        # Calculate the perspective transform matrix
        M = cv2.getPerspectiveTransform(src_points, dst_points)

        # Apply the perspective warp to get the cropped image
        warped = cv2.warpPerspective(image_np, M, (width, height))

        # Convert the cropped image back to PIL format
        cropped_image = Image.fromarray(warped)
        cropped_images.append(cropped_image)

    return cropped_images

In [ ]:
cropped_query_image_list_d1 = []
pbar = tqdm(zip(sorted(QSD1_W4_PATH.glob("*.jpg")), denoised_query_d1_PIL_list, FRAMES_QSD1_W4_LIST))
for name, image, predicted_mask in pbar:
    pbar.set_description(f"Splitting {name.stem}")
    cropped_query_image_list_d1.append(crop_paintings_from_image(image, predicted_mask))

In [ ]:
for l in cropped_query_image_list_d1:
    for img in l:
        #display(img)
        pass

## Keypoint extraction

In [ ]:
from src.descriptors import SIFTDescriptor, ORBDescriptor, HOGDescriptor, ImageRetrievalSystem

In [ ]:
descriptors = {
    'SIFT': SIFTDescriptor(),
   # 'ORB': ORBDescriptor(),
   # 'HOG': HOGDescriptor()
}
descriptor_path = WEEK_4_RESULTS_PATH / 'Task_1' / 'descriptors'  # Directory to save/load descriptor files

# Instantiate the image retrieval system
retrieval_system = ImageRetrievalSystem(descriptors, descriptor_path)

In [ ]:
# Set number of top similar images to retrieve
K = 10

# Retrieve similar images using each descriptor separately
for descriptor_name in ['SIFT']:
    print(f"\nRetrieving similar images using {descriptor_name} descriptor:")
    results, g_0 = retrieval_system.retrieve_similar_images(cropped_query_image_list_d1, database_image_PIL_list, K, descriptor_name)

In [ ]:
from src.metrics import MeanAveragePrecisionAtK

In [ ]:
map = MeanAveragePrecisionAtK()

In [ ]:
GT_mod = []
for element in GT_QSD1_W4_LIST:
    if len(element) == 2:
        GT_mod.append([element[0]])
        GT_mod.append([element[1]])
    else:
        GT_mod.append([element[0]])

results_mod = []
for element in results:
    if len(element) == 2:
        results_mod.append(element[0])
        results_mod.append(element[1])
    else:
        results_mod.append(element[0])

print(GT_mod)
print("#")
print(results_mod)

In [ ]:
result = map.compute(GT_mod, results_mod, k=K)
print(result)

In [ ]:
idx = 0
non_found_dist = []
for res, gt, gap in zip(results_mod, GT_mod, g_0):
    print(f"{idx} | Res: {res}, GT: {gt}, gaps: {gap[1]}")
    idx += 1

In [ ]:
db_i = descriptors['SIFT'].compute(np.array(database_image_PIL_list[32]))
query_i = descriptors['SIFT'].compute(np.array(cropped_query_image_list_d1[4][0]))
print(db_i[0][0])

In [ ]:
from scipy.spatial.distance import cdist

distances = cdist(db_i[1], query_i[1], metric='euclidean')
min_distances = np.min(distances, axis=1)
mean = np.mean(min_distances)

print(mean)

db_i = descriptors['SIFT'].compute(np.array(database_image_PIL_list[177]))
distances = cdist(db_i[1], query_i[1], metric='euclidean')
min_distances = np.min(distances, axis=1)
mean = np.mean(min_distances)

print(mean)
